In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

pd.options.display.max_columns = 10 
pd.options.display.width = 134
pd.options.display.max_rows = 20

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



In [18]:
data_folder = '../Data_contest/dataset/'


genome_scores_df=pd.read_csv(data_folder+'genome_scores.csv') # Large (500MB)
movies_df=pd.read_csv(data_folder+'movies.csv')
validation = pd.read_csv(data_folder+'validation.csv') 

train=pd.read_csv(data_folder+'train_with_valid.csv') # Large 500MB
test=pd.read_csv(data_folder+'test.csv') # Large 500MB


In [23]:
A=train.movieId
len(A)

5393573

In [16]:
train.pivot('userId','movieId','rating')

movieId,0,1,2,3,4,...,8354,8355,8358,8359,8360
userId,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN
2,4.0,3.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
4,4.0,3.5,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
7,3.0,2.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
9,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
10,5.0,5.0,2.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN
11,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [13]:

matrix = pd.concat([train,test]).pivot('userId','movieId','rating')
matrix = matrix.drop_duplicates(subset = ['userId','movieId'],keep = 'first')
movie_means = matrix.mean()
user_means = matrix.mean(axis=1)
mzm = matrix-movie_means
mz = mzm.fillna(0)
mask = -mzm.isnull()



/home/rahulchakwate/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


ValueError: Index contains duplicate entries, cannot reshape

In [4]:
iteration = 0
mse_last = 999
while iteration<10:
    iteration += 1
    svd = TruncatedSVD(n_components=20,random_state=42)
    svd.fit(mz)
    mzsvd = pd.DataFrame(svd.inverse_transform(svd.transform(mz)),columns=mz.columns,index=mz.index)

    mse = mean_squared_error(mzsvd[mask].fillna(0),mzm[mask].fillna(0))
    print('%i %.5f %.5f'%(iteration,mse,mse_last-mse))
    mzsvd[mask] = mzm[mask]

    mz = mzsvd
    if mse_last-mse<0.00001: break
    mse_last = mse

m_20 = mz+movie_means
m_20 = m_20.clip(lower=0.5,upper=5)


1 0.03400 998.96600
2 0.03149 0.00251
3 0.03022 0.00127
4 0.02943 0.00079
5 0.02888 0.00055
6 0.02847 0.00041
7 0.02816 0.00032
8 0.02790 0.00026
9 0.02769 0.00021
10 0.02751 0.00018


In [5]:
iteration = 0
mse_last = 999
while iteration<10:
    iteration += 1
    svd = TruncatedSVD(n_components=15,random_state=42)
    svd.fit(mz)
    mzsvd = pd.DataFrame(svd.inverse_transform(svd.transform(mz)),columns=mz.columns,index=mz.index)

    mse = mean_squared_error(mzsvd[mask].fillna(0),mzm[mask].fillna(0))
    print('%i %.5f %.5f'%(iteration,mse,mse_last-mse))
    mzsvd[mask] = mzm[mask]

    mz = mzsvd
    if mse_last-mse<0.00001: break
    mse_last = mse

m_15 = mz+movie_means
m_15 = m_15.clip(lower=0.5,upper=5)


1 0.02860 998.97140
2 0.02841 0.00019
3 0.02826 0.00014
4 0.02814 0.00012
5 0.02804 0.00010
6 0.02795 0.00009
7 0.02788 0.00008
8 0.02780 0.00007
9 0.02774 0.00006
10 0.02768 0.00006


In [7]:
#bagging
m = (m_15+m_20)/2

In [8]:
df_best_tillnow = pd.read_csv("./Submission_pq_regression_neighbour_baseline.csv")

In [9]:

test['rating'] = test.apply(lambda x:m[m.index==x.userId][x.movieId].values[0],axis=1)

# There are some movies who did not have enough info to make prediction, so just used average value for user
# missing = np.where(test.rating.isnull())[0]
# test.ix[missing,'rating'] = user_means[test.loc[missing].userId].values

indices=np.where(test.rating.isna()) 
test.rating.iloc[indices] = df_best_tillnow.Prediction.iloc[indices]

# test['rating'] = np.around(test['rating'],1)
test.to_csv('submission_truncSVD_15_20_25.csv',index=False)


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [39]:
df_chakki = pd.read_csv("./emsemble_models/Submission_truncSVD20_15_correct.csv")
df_latent = pd.read_csv("./emsemble_models/Submission_pq.csv")
# df_latent_old = pd.read_csv("./emsemble_models/Submission_pq_rahul_old.csv")
# df_latent_3dim = pd.read_csv("./emsemble_models/Submission_pq_3dim.csv")
df_regression = pd.read_csv("./emsemble_models/Submission_regression_final.csv")
df_nbhood =  pd.read_csv("./emsemble_models/Submission_nbhood_vishwesh.csv")
df_nn =  pd.read_csv("./emsemble_models/Submission_regression_NN.csv")


PRED = (8*df_chakki.Prediction + df_latent.Prediction + df_regression.Prediction + 4*df_nn.Prediction)/14
PRED = np.around(PRED,1)


In [40]:
df_test=pd.read_csv(data_folder+'test.csv') # Large 500MB

predictions = np.zeros(len(df_test))
for i in range(len(df_test)):
    userid =  df_test.iloc[i,0]
    movieid = df_test.iloc[i,1]
    rating = PRED[i]
    if rating>5:
        rating = 5
    if rating<0.5:
        rating =0.5
    predictions[i] = rating
df_submission = pd.read_csv(data_folder+'dummy_submission.csv')
df_submission.Prediction = predictions
df_submission.to_csv('./emsemble_models/submission_emsembling_21nn.csv',index=False)

In [48]:
a = np.array([3.446,12.133])
# a_s = float("{0:.1f}".format(3.553))
b=np.around(a,1)
b

array([ 3.4, 12.1])

In [ ]:
test['rating'] = float("{0:.1f}".format(test['rating']))

In [22]:
df_chakki = pd.read_csv("./submission_truncSVD2.csv")

In [23]:
df_chakki = df_chakki.drop(['Unnamed: 0'],axis=1)

In [24]:
df_chakki.columns

Index(['ID', 'rating'], dtype='object')

In [25]:
df_chakki.rating.isna().sum()

305688

TypeError: Index does not support mutable operations

In [26]:
df_chakki.rename(columns={'rating':'Prediction'}, inplace=True)

In [27]:
df_chakki.head()

,ID,Prediction
0,NaN,3.892154
1,NaN,3.308117
2,NaN,2.410003
3,NaN,3.933171
4,NaN,3.383848


In [28]:
df_chakki.to_csv("./Submission_truncSVD2.csv",index=False)